In [1]:
import os 
import string
from numpy import loadtxt
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings('ignore')
import sys, os
sys.path.append(os.pardir)
import glob
import numpy as np
from multiprocessing import Pool
import multiprocessing as multi
sys.path.append(os.pardir)
import pandas as pd
import h5py
import re
import gensim


# lines = loadtxt("/home/louis/SharedWindows/edu_data/"+"imdb-edus.train")
# train = []

# with open(dir+"imdb-edus.train", 'r') as train_file:
#     lines = train_file.readlines()


##



# '''text2index'''
# 
# from func import TextDeal
# from func import wrapper_func_four
# 
# vocab = wList
# 
# td = TextDeal()
# load_path = sorted( glob.glob( '/home/dl-box/デスクトップ/PythonFile/hirose/Text/text_list/*.txt' ) )
# name = [ os.path.splitext( os.path.basename( r ) )[0] for r in load_path ]
# save_path = '/home/dl-box/デスクトップ/PythonFile/hirose/LSTM/W2I/'



/home/owner/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/owner/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [13]:

##

def wrapper_func_six(tuple_data):
    return(tuple_data[0](tuple_data[1],tuple_data[2],tuple_data[3],tuple_data[4], tuple_data[5], tuple_data[6]) )

def readEduStyledFiles(dir, filename):
    with open(dir+filename, 'r') as train_file:
        lines = train_file.readlines()
    return lines
        
def tokenizeListOfTexts(list, vocab):
    idx = []
    for word in list:
        if word in vocab:
            idx.append(vocab[word].index)
            #print(word)
        # else:
        #     print(word)
    return idx


def cleanAndTokenizeEDUs(vocab, load_path=None, load_name=None, save_path=None, save_name=None, willReturn = True, maxlen = 15):
    
    if (willReturn == False) and (all(v is None for v in [load_path, load_name, save_path, save_name])):
        raise ValueError('Function Arguments cannot all be None Type when saving files')
    elif (willReturn == True) and (all(v is None for v in [load_path, load_name])):
        raise ValueError('Load arguments cannot all be None Type when loading file')
    
    lines = readEduStyledFiles(load_path,load_name)

    cleaned_lines = []
    length = len(lines)
    scores = []
    ids = []
    cleaned_doc=[]
    cleaned_all=[]
    idx_binary = [] # index for binary training only
    doc_count = -1
    
    for i in range(len(lines)):
        cleaned = lines[i].replace('\n','').replace('<s>','').replace("'",'')
        cleaned = text_to_word_sequence(cleaned)
        
        
        if (len(cleaned) == 2): # and (i != 0):
            if  (cleaned[0].isnumeric() and cleaned[1].isnumeric()):
                score = int(cleaned[0])
                id = int(cleaned[1])
                if (score<=4) or (score>=7):
                    idx_binary.append(doc_count)
                filtered_doc = list(filter(None, cleaned_doc))
                padded_doc = pad_sequences(filtered_doc, maxlen=maxlen)
                scores.append(score)
                ids.append(id)
                cleaned_all.append(padded_doc)
                cleaned_doc=[]
                doc_count += 1
                continue
        
        tokenized=tokenizeListOfTexts(cleaned, vocab)
        
        if (i%1000000 == 0) and (i!=0):
            print(str(i)+' out of' + str(length))
            #break
            # print(cleaned_lines[i-500:i])
    
        cleaned_doc.append(tokenized)
        # cleaned_doc = cleaned_doc[0]
    
    del cleaned_all[0]
    del scores[0]
    del idx_binary[0]
    del ids[0]
    features = np.asarray(cleaned_all)
    labels = np.asarray(scores)
    idx = np.asarray(idx_binary)
    
   
    
    if (willReturn == True):
        print('cleanAndTokenizeEDUs Returned and Done')
        return features, labels, idx, ids
    else:
        np.save(save_path + save_name + '_features'+ '.npy', features)
        np.save(save_path + save_name + '_labels' + '.npy', labels)
        np.save(save_path + save_name + '_idx_binary_only' + '.npy', idx)
        np.save(save_path + save_name + '_doc_ids' + '.npy', ids)
        print(save_name + ' cleanAndTokenizeEDUs Saved and Done')

dir = '/home/louis/SharedWindows/edu_data/'  

def getWordIndices()
# t = Tokenizer()

# tokenized = t.fit_on_texts(cleaned)


In [14]:
if __name__ == '__main__':
    #data=(関数, 引数)
    isReload = int(input("Type 1 to initiate reload of google word2vec: "))
    
    if isReload:
        model_word2vec_temp = gensim.models.KeyedVectors.load_word2vec_format('/home/owner/デスクトップ/milnet+edu/GoogleNews-vectors-negative300.bin', binary=True)  
        model_word2vec = model_word2vec_temp

    print('word2vec model loaded')

    load_path = '/home/owner/デスクトップ/milnet+edu/data/'  
    save_path = '/home/owner/デスクトップ/milnet+edu/data/Preprocessed/'
      
    load_name = ['imdb-edus.train', 'imdb-edus.test','imdb-edus.dev']
    save_name = ['train_data','test_data','validation_data']
    
    willReturn = False
    
    # cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = 'imdb-edus.train', save_path=save_path,save_name=save_name[0], willReturn = False)
    # features, scores, idx, ids = cleanAndTokenizeEDUs(model_word2vec.vocab, load_path=load_path, load_name = 'imdb-edus.train', save_path=save_path,save_name=save_name[0], willReturn = False)
    
    vocab = model_word2vec.vocab
    data = [ ( cleanAndTokenizeEDUs, vocab, load_path, load_name[i], save_path, save_name[i] , willReturn) for i in range( len(load_name) ) ]
    with Pool( multi.cpu_count()-1 ) as p:
        p.map( wrapper_func_six, data )
        
    print('All Done')



Type 1 to initiate reload of google word2vec: 0
word2vec model loaded
1000000 out of11039529
1000000 out of1349298
1000000 out of1335054
test_data cleanAndTokenizeEDUs Saved and Done
2000000 out of11039529
validation_data cleanAndTokenizeEDUs Saved and Done
3000000 out of11039529
4000000 out of11039529
5000000 out of11039529
6000000 out of11039529
7000000 out of11039529
8000000 out of11039529
9000000 out of11039529
10000000 out of11039529
11000000 out of11039529
train_data cleanAndTokenizeEDUs Saved and Done
All Done


In [11]:
test_labels = np.load('/home/owner/デスクトップ/milnet+edu/data/Preprocessed/test_data_labels.npy')

In [12]:
print(test_labels[0])

0


NameError: name 'avg' is not defined

In [15]:
model_word2vec_temp.wv.syn0

array([[ 1.1291504e-03, -8.9645386e-04,  3.1852722e-04, ...,
        -1.5640259e-03, -1.2302399e-04, -8.6307526e-05],
       [ 7.0312500e-02,  8.6914062e-02,  8.7890625e-02, ...,
        -4.7607422e-02,  1.4465332e-02, -6.2500000e-02],
       [-1.1779785e-02, -4.7363281e-02,  4.4677734e-02, ...,
         7.1289062e-02, -3.4912109e-02,  2.4169922e-02],
       ...,
       [-1.9653320e-02, -9.0820312e-02, -1.9409180e-02, ...,
        -1.6357422e-02, -1.3427734e-02,  4.6630859e-02],
       [ 3.2714844e-02, -3.2226562e-02,  3.6132812e-02, ...,
        -8.8500977e-03,  2.6977539e-02,  1.9042969e-02],
       [ 4.5166016e-02, -4.5166016e-02, -3.9367676e-03, ...,
         7.9589844e-02,  7.2265625e-02,  1.3000488e-02]], dtype=float32)